In [2]:
"""
Calculate steady-state solutions for a combustor, modeled as a single well-stirred
reactor, for different residence times.

We are interested in the steady-state burning solution. This example explores
the effect of changing the residence time on completeness of reaction (through
the burned gas temperature) and on the total heat release rate.

Demonstrates the use of a MassFlowController where the mass flow rate function
depends on variables other than time by capturing these variables from the
enclosing scope. Also shows the use of a PressureController to create a constant
pressure reactor with a fixed volume.

Requires: cantera >= 2.5.0, matplotlib >= 2.0
Keywords: combustion, reactor network, well-stirred reactor, plotting
"""

import numpy as np
import matplotlib.pyplot as plt
import cantera as ct

# Use reaction mechanism GRI-Mech 3.0. For 0-D simulations,
# no transport model is necessary.
gas = ct.Solution('./Jerzembeck.cti')

# Create a Reservoir for the inlet, set to a methane/air mixture at a specified
# equivalence ratio
equiv_ratio = 0.884043442  # lean combustion
gas.TP = 800.0, 30*ct.one_atm
gas.set_equivalence_ratio(equiv_ratio, 'IXC8H18:1.0', 'O2:1.0, N2:3.76')
inlet = ct.Reservoir(gas)

# Create the combustor, and fill it initially with a mixture consisting of the
# equilibrium products of the inlet mixture. This state corresponds to the state
# the reactor would reach with infinite residence time, and thus provides a good
# initial condition from which to reach a steady-state solution on the reacting
# branch.
gas.equilibrate('HP')
combustor = ct.IdealGasReactor(gas)
combustor.volume = 0.4

# Create a reservoir for the exhaust
exhaust = ct.Reservoir(gas)

# Use a variable mass flow rate to keep the residence time in the reactor
# constant (residence_time = mass / mass_flow_rate). The mass flow rate function
# can access variables defined in the calling scope, including state variables
# of the Reactor object (combustor) itself.


def mdot(t):
    return combustor.mass / residence_time


inlet_mfc = ct.MassFlowController(inlet, combustor, mdot=mdot)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(combustor, exhaust, master=inlet_mfc, K=0.01)

# the simulation only contains one reactor
sim = ct.ReactorNet([combustor])

# Run a loop over decreasing residence times, until the reactor is extinguished,
# saving the state after each iteration.
states = ct.SolutionArray(gas, extra=['tres'])

residence_time = 0.1  # starting residence time
while combustor.T > 500:
    sim.set_initial_time(0.0)  # reset the integrator
    sim.advance_to_steady_state()
    print('tres = {:.2e}; T = {:.1f}'.format(residence_time, combustor.T))
    states.append(combustor.thermo.state, tres=residence_time)
    residence_time *= 0.9  # decrease the residence time for the next iteration

# Plot results
f, ax1 = plt.subplots(1, 1)
ax1.plot(states.tres, states.heat_release_rate, '.-', color='C0')
ax2 = ax1.twinx()
ax2.plot(states.tres[:-1], states.T[:-1], '.-', color='C1')
ax1.set_xlabel('residence time [s]')
ax1.set_ylabel('heat release rate [W/m$^3$]', color='C0')
ax2.set_ylabel('temperature [K]', color='C1')
f.tight_layout()
plt.show()


/var/folders/cf/lk26ttt55ds1sl3wv1t4lfs00000gn/T/ipykernel_29008/532782868.py:24: DeprecationWarning: XML_Node::build: 
The CTI and XML input file formats are deprecated and will be removed in
Cantera 3.0. Use 'cti2yaml.py' or 'ctml2yaml.py' to convert CTI or XML input
files to the YAML format. See https://cantera.org/tutorials/legacy2yaml.html
for more information.
  gas = ct.Solution('./Jerzembeck.cti')


tres = 1.00e-01; T = 2531.4
tres = 9.00e-02; T = 2531.5
tres = 8.10e-02; T = 2531.5
tres = 7.29e-02; T = 2531.5
tres = 6.56e-02; T = 2531.6
tres = 5.90e-02; T = 2531.6
tres = 5.31e-02; T = 2531.7
tres = 4.78e-02; T = 2531.8
tres = 4.30e-02; T = 2531.8
tres = 3.87e-02; T = 2531.9
tres = 3.49e-02; T = 2532.0
tres = 3.14e-02; T = 2532.0
tres = 2.82e-02; T = 2532.1
tres = 2.54e-02; T = 2532.2
tres = 2.29e-02; T = 2532.3
tres = 2.06e-02; T = 2532.4
tres = 1.85e-02; T = 2532.5
tres = 1.67e-02; T = 2532.7
tres = 1.50e-02; T = 2532.8
tres = 1.35e-02; T = 2532.9
tres = 1.22e-02; T = 2533.0
tres = 1.09e-02; T = 2533.2
tres = 9.85e-03; T = 2533.3
tres = 8.86e-03; T = 2533.4
tres = 7.98e-03; T = 2533.6
tres = 7.18e-03; T = 2533.7
tres = 6.46e-03; T = 2533.8
tres = 5.81e-03; T = 2533.9
tres = 5.23e-03; T = 2534.0
tres = 4.71e-03; T = 2534.1
tres = 4.24e-03; T = 2534.1
tres = 3.82e-03; T = 2534.2
tres = 3.43e-03; T = 2534.2
tres = 3.09e-03; T = 2534.1
tres = 2.78e-03; T = 2534.1
tres = 2.50e-03; T =

CanteraError: 
*******************************************************************************
CanteraError thrown by CVodesIntegrator::step:
CVodes error encountered. Error code: -3
Exceptions caught during RHS evaluation:

*******************************************************************************
CanteraError thrown by checkFinite:
ydot contains non-finite elements:

ydot[2] = nan
ydot[8] = inf
ydot[20] = inf
ydot[22] = nan
ydot[23] = -inf
ydot[33] = -inf
ydot[52] = nan
ydot[87] = inf
*******************************************************************************


*******************************************************************************
CanteraError thrown by checkFinite:
ydot contains non-finite elements:

ydot[2] = nan
ydot[3] = inf
ydot[5] = inf
ydot[8] = -inf
ydot[9] = -inf
ydot[11] = inf
ydot[14] = inf
ydot[15] = inf
ydot[17] = inf
ydot[18] = -inf
ydot[19] = inf
ydot[20] = inf
ydot[22] = -inf
ydot[23] = inf
ydot[33] = -inf
ydot[51] = -inf
ydot[52] = -inf
ydot[86] = i